# Project Title: Predicting Flight Delays through Machine Learning Classifiers at Scale

Our objective is to help airlines better allocate resources and make business decisions through improved flight delay predictions.

# Group20 - Phase 2 Deliverable

#### Phase Leader Plan

| Phase | Leader | Description |
| --- | --- | --- |
| 1 | Dominic Lim | Project Plan, describe datasets, joins, tasks, and metrics |
| 2 | Nathan Chiu | EDA, baseline pipeline on all data, Scalability, Efficiency, Distributed/parallel Training, and Scoring Pipeline |
| 3 | Raul Merino | Feature engineering + hyperparameter tuning, + in-class review |
| 4 | Javier Rondon | Advanced model architectures and loss functions, select an optimal algorithm, fine-tune & Final report write up  |

#### Credit Assignment

| Task                           | Owner       | Hours (estimate) |
|--------------------------------|-------------|------------------|
| Video                          |Nathan team  |             2    |               
| Review proofreading            | Dominic, Javier, Nathen, Raul|5|
| EDA - Weather                   | Javier |10|
| EDA - Flights                   |  Dominic |10|
| Baseline, Cross Validation and Split             | Raul |5|       
| Join databases        | Raul Nathan |15|
| Pipeline preparation |  Raul |5|

#### Sources used throughout this notebook:
- Peterson, Everett B., et al. “The Economic Cost of Airline Flight Delay.” Journal of Transport Economics and Policy, vol. 47, no. 1, 2013, pp. 107–21. JSTOR, http://www.jstor.org/stable/24396355. Accessed 30 Oct. 2022.
- Aviation Data & Statistics. Aviation Data & Statistics | Federal Aviation Administration. (n.d.). Retrieved October 30, 2022, from https://www.faa.gov/data_research/aviation_data_statistics 
- J. Rondon personal communication, October 27, 2022
- https://www.sciencedirect.com/topics/engineering/wet-bulb-temperature 
- https://education.nationalgeographic.org/resource/barometer
- https://cardinalscholar.bsu.edu/bitstream/handle/123456789/200785/Algarin%20BallesterosJ_2017-3_BODY.pdf?sequence=1&isA llowed=n
- https://engineering.berkeley.edu/news/2010/11/flight-delays-cost-more-than-just-time/
- https://www.airlines.org/dataset/u-s-passenger-carrier-delay-costs/

https://www.quora.com/How-are-the-outer-runways-of-DFW-used-in-conjunction-with-the-others
https://i0.wp.com/worldairlinenews.com/wp-content/uploads/2015/03/lga-airport-map-faalr-copy.jpg?resize=600%2C489&ssl=1
https://www.skyscanner.com/tips-and-inspiration/what-windspeed-delays-flights
https://www.faa.gov/about/office_org/headquarters_offices/ato/service_units/techops/navservices/lsg/rvr

## Section 1: Abstract

Delays in commercial aviation are frequent and expensive. With roughly 20% of all flights being categorized as being delayed by more than 15 minutes, the downstream costs to the airlines and passengers can total tens of billions of dollars annually. Utilizing Flight Data from the Bureau of Transportation Statistics and Weather Data from the National Oceanic and Atmospheric Administration, our Team is tasked to predict U.S. domestic flight delays two hours before the scheduled departure time.

Our primary use case for flight delay prediction is Airline resource planning and allocation. With only a two-hour notice, airlines would be unable to reposition planes or pilots as contingencies to deal with irregular operations must be put in place 72 hours before departure. However, airlines would be able to allocate corporate resources in anticipation of the fallout from delays (i.e., surge resourcing for help and call centers).

Once we have transformed the data after an initial Exploratory Data Analysis, we plan on joining our datasets utilizing composite keys, with the ultimate goal of using machine learning algorithms such as Decision Trees, Random Forests, and Neural Networks to predict flight delays. We plan on evaluating the performance of our model by comparing the F-2 metric of our machine learning models and a baseline model. Based on our EDA, we decided that our baseline model would simply predict that no delays would take place.

## Section 2: Data Description (Include Visualizations)

##### Flight Data

The primary data source is the Flight Dataset from the Bureau of Transportation Statistics. The data covers flights from 2015 to 2021 and includes 109 features. The Flight Data provides us with the 2 potential response variables:

- `DEP_DEL15` :  Classification of the flight being delayed by greater than 15 minutes
- `DEP_DELAY`:  Minutes elapsed between Scheduled and Actual Departure Time

The dataset includes both numerical and categorical features that could be useful predictor variables. Numerical features include the time (`CRS_DEP_TIME`), date (`FL_DATE`) and distance flown for the flights. Categorical features such as Airline Carrier (`OP_CARRIER`), Plane Identifier (`TAIL_NUM`) are also of interest.

We conducted an exploratory data analysis of the 3-month sample flight dataset covering the 1st quarter of 2015 (2.8 million recorded flights). The EDA was conducted with a special focus on computing % of missing values per feature, understanding the distribution, scale and range of values of the features.

As a first step, we decided to drop any features with more than 50 percent of missing values from the dataset. The resulting dataset shrank the dimensions of the dataset from 109 features to 56 features. Of note, 49 of the dropped features were primarily related to flights diverted away from scheduled airports (99.74% missing values). We also decided to filter out cancelled flights that did not computed minutes in delay.

Table 1 Missing Values from Flight Data

<img src="https://drive.google.com/uc?export=view&id=1k8ZVVr6Rsmg6NRzcGDtqAssJ2fskefHk" alt="Google Drive Image" width=30%/>

From calculating the proportion of the Delayed and Not-Delayed classes from the predictor variable, `DEP_DEL15`, we observe that the majority of flights are classified as "not delayed". Our sample dataset corroborates our literature review that approximately 20% of all flights are delayed.

Table 2 Proportion of Flights that are Delayed (≥ 15 minutes)

|DEP_DEL15|cnt_per_group|perc_of_count_total|
|---------|-------------|-------------------|
|      0.0|      2166530|  79.58653046470691|
|      1.0|       555702| 20.413469535293096|

We observe from the distribution of `DEP_DELAY` that amongst flights with delays, the majority of delays are less than 30 minutes. It is important to note that negative values are computed for flights that depart early than scheduled. 

Figure 1 Distribution of flight delays in minutes

<img src="https://drive.google.com/uc?export=view&id=1yUw1L3z5efGxp6rv0lLNpIdR56Qk-9_7" alt="Google Drive Image" width=30%/>

Amongst the predictor variables in our dataset, we are particularly interested in the scheduled departure time (`DEP_TIME_BLK`), scheduled departure day of week (`DAY_OF_WEEK`), and Airlines Carrier (`OP_CARRIER`). We parsed the scheduled-departure hour of delayed flights and we observe that as the day progresses, the percentage of flights that are delayed steadily increase. We can speculate that there are network effects whereby earlier delayed flights affect later scheduled flight. This might be a result of delayed scheduled flights taking up Airport Terminal and Gate capacity.

Figure 2 Distribution of flight delays by Scheduled Departure hour

<img src="https://drive.google.com/uc?export=view&id=1D35X80alBoG3aM1N-TUi57aJBP1hSyp0" alt="Google Drive Image" width=50%/>

As it relates to the day-of-the-week of scheduled flights, we observe upticks in flight delays as a % of total flights on Monday, Thursday, Friday, and Sunday. This may be a result of increased  demand and stress on the Airline/Airport systems on days when passengers are more likely to be traveling for business (Monday - Thu/Friday) split and weekend travellers departing and returning from a trip (Friday/Sunday). 

Figure 3 Distribution of flight delays by "Day of Week"

<img src="https://drive.google.com/uc?export=view&id=15-jlbGqaswTMHUNzD4K4Z5lEABZz9w4o" alt="Google Drive Image" width=30%/>

Airline Carrier is a notable categorical variable and is of particular interest. We can see a substantial range in flight timeliness performance by Airline Carrier. The worst performing Airline, Frontier Airlines (`F9`) is followed by Envoy Airlines/American Eagle (`MQ`) and JetBlue (`B6`). An interesting problem that we may need deal with is Airline Carriers that consolidate via mergers/acquisitions or are out of business.

Figure 4 Distribution of flight delays by Airline Carrier

<img src="https://drive.google.com/uc?export=view&id=1Yl5G9j3wyYda_afJ1oZ65deSjDTZ9d4T" alt="Google Drive Image" width=30%/>

d

#### Weather Data

The second data source is the Local Climatological Data (LCD) dataset from the National Centers for Environmental Information. The data covers the period from January 2015–December 2021). The LCD data contains summaries of climatological conditions from weather stations managed by the NWS, FAA, and DOD. Since the weather impacts airline flight operations, it is reasonable to assume that weather features may have predictive power in our model for flight delays.

We conducted an exploratory data analysis (EDA) on the weather set to spot anomalies, calculate the number of missing values, understand the distribution, scale, and range of values of the features, and propose how the features may have explanatory value for our model.

 The data includes hourly observations of temperature, humidity, wind direction and speed, barometric pressure, sky condition, visibility, and weather phenomena. These are critical observations of the atmosphere that help forecasters predict the weather.  

We reviewed the contents of the LCD dataset covering the period January-2015- December 2021. The observations correspond to 15079 weather stations. Overall the dataset contains 124 columns and approximately 900 million rows. We filtered the observations to include only weather stations located at airports in the US and territories. After this reduction, the dataset contained weather observations for 379 weather stations with 35 million rows and 26 columns.

We reviewed the contents of the weather observations to understand the contents, units of observation and report types. The dataset contained observations from seven differnt types of reports. Based on the report type, we decided to only allow observations from reports taken at hourly frequency (FM-15) and special weather reports (FM-16). This further reduced the nunber of rows to 31 million.

We estimated the number of missing values from any of the weather features in the model. Table 1 shows the dataset's features and the number of missing values. Table. shows the data types in the raw data. We reviewed each weather feature to assess their usefulness in explaining weather delays. THe pressence of many zeroes or missing values does not mean the feature has to be discarted.

**Table 1 Missing Values from weather data** \
<img src="https://drive.google.com/uc?export=view&id=1e2GVFKPLgwbTI_lvGByiqlcXYXqESSA9" alt="Google Drive Image" width=30%/>

**Table 2. Data dictionary from weather data after removal of features** \
<img src="https://drive.google.com/uc?export=view&id=1qIce1wYJ9ERAWB5ONOg94Hupx9MZHz9Z" alt="Google Drive Image" width=30%/>


We want to use weather features that are typical causes for weather delays. According to the literature weather delays are related to the presence of thunderstorms, Ice, Freezing rain, snow and fog. These events may impact airport runwayt capacity, traffic control and ground operations and cause delays. The features in the weather data may be able to be used to predict delays at airports.

#### Precipitation

The feature 'HourlyPrecipitation' contains the values of liquid precipitation observation values given in inches to hundredths. Traces amounts are reported as T. Null or blank values indicate no precipittion was observed. We replaced all the null values and "T" values with zero.  values with "S" indicate suspect values. These values accout for only 0.1 percent of the data. Some of these suspect values correspond to very high amounts of precipitation per hour recorded, exceding 9 inches of rain per hour. These extreme events may be anomalies in the data or correspond to rare meteorological events. Overall 92 percent of the values in the "HourlyPrecipitation" feature are zero. We retain this feature because the amount of precipitation is likely related to weather delays at airports. Having such a large percentage of zeroes is expected since not all locations get rain every month or season.

#### Temperature

The weather data set contains three features for temperature observations, wet bulb, dry bulb, and dew point temperature. The difference between wet bulb and dry bulb temperatures is a measure of the humidity of the air. The higher the difference in these temperatures, the lower the humidity. The dew point is the temperature under which water vapor condenses. The higher the dew point, the higher the moisture in the air. Figure 1 shows a statistical summary of the three temperature features. The units of temperature are in degrees Fahrenheit.

**Figure 1 Statistical summary temperature features** \
<img src="https://drive.google.com/uc?export=view&id=1OHDGBSTRSLTifw6dsi3zY0VC1aZQ8UI4" alt="Google Drive Image" width=70%/>

Figure 2 shows an example of the yearly and daily variation of the temperature. The left panel shows the seasonal variation of the reported temperatures reported by the Dallas-Fort Worth airport (DFW). The right panel shows the hourly variation of the temperature at this airport during a day (7-4-2015).  Changes in temperature may be indicative of weather events such as cold fronds, storms that may impact airport operations and cause delays.

**Figure 2 Daily and average temperatures reported by the DFW airport weather station** \
<img src="https://drive.google.com/uc?export=view&id=10jnC7A3wUc2-Mt-XI8ync9qq5p21iLT2" alt="Google Drive Image" width=70%/>

#### Pressure

Figure 2 shows a statistical summary of the pressure features in the data set. The 'HourlyStationPressure' is the atmospheric pressure observed at the station and reported in inches of Mercury (in Hg). This feature has 2 percent of missing values. The range of values seems reasonable, with the lowest pressure reported approximately 9 psi, corresponding to higher altitude stations or weather events. 

The 'HourlyPressureTendency' indicates increases or decreases in pressure over the 3 previous hours. The 'HourlyPressureChange' indicates a difference in pressure over the past 3 hours. Rapid drops in atmospheric pressure measurements are associated with cloudy, rainy, or windy weather, while rapid increases are associated with clear skies. This feature could be valuable for our model however 77 percent of the values for these features are missing and they will not be introduced in our model. However we can estimate a similar feature using the 'HourlyStationPressure' to estimate the change of pressure over time and introduce that in the model.

**Figure 2 Statistical summary pressure features** \
<img src="https://drive.google.com/uc?export=view&id=1MohYdajBcUxgvEp3bysRt4Vf91hhnQoh" alt="Google Drive Image" width=70%/>

#### Wind

Figures 3 and 4 show the statistical summary of the wind-related features in the data set. Strong surface winds, such as crosswinds and gusts, can impact aircraft coming to land or takeoff, causing disruptions to airport operations. While operations depend on runway length and aircraft size, in general crosswinds in excess of 30-35 kts (about 34-40 mph) are generally prohibitive of take-off and landing.


Some anomalous high wind speed values were removed, and most observations are less than ten mph winds. The most significant frequency for wind direction was less than 36 degrees. 

**Table 3  Statistical summary of wind features** \
<img src="https://drive.google.com/uc?export=view&id=1DJnTI3BgsOELhKR6x5X946vz-sADELKf" alt="Google Drive Image" width=50%/>

To review the relationship of wind direction with delays we reviewed the distribution of the wind direction at airports. Figure 3 shows an example of the distribution of the winds reported at the DFW airport during 2015 and the runway configuration of this airport. The runway configuration was built considering the prevailing winds, to reduce the impacts of crosswinds on the runways which can cause delays. This indicates that to use the wind direction as a feature that can cause flight delays, the individual airport configurations must be considered. In Figure 3, we can observe that the runwas are placed in an orientation compatible with the prevailing winds. The only wind directions that may affect airport operations occurr at a very low frequency  (45 and 90 degrees)

**Figure 3 Distribution of wind direction at DFW airport and runway configuration** \
<img src="https://drive.google.com/uc?export=view&id=1KqxAnLtgkY2vLUVa6dt7PzIoNu-AIoI7" alt="Google Drive Image" width=70%/>

Figure 4 shows a similar situation with LGA airport. In the figure we can see that the orientation of the runways at this airport is compatible with the prevailing winds in the area. The perpendicular runways at this airport leverage the direction of the prevailing winds, therefore the wind direction impact is lower.  From this review we believe that we can use the wind direction as a feature indicating when there are significant crosswinds at the airport. We can obtain runway orientation information for the US airports and attempt to use it as a feature in our model.

**Figure 4 Distribution of wind direction at LGA airport and runway configuration** \
<img src="https://drive.google.com/uc?export=view&id=1GUv1Oab2HMh_fbNbZ8IWhv_1YDtk7ZQ1" alt="Google Drive Image" width=70%/>


Figure 5 shows the hourly wind and wind gusts speeds for the years 2015-2021 in the DFW airport.Since in general crosswinds in excess of about 34-40 mph are restrictive of take-off and landing, the use of these two features in adddition with the wind orientation may have explanatory power for some delays related to weather.

**Figure 5 Distribution of wind and wind gust speeds at DFW airport** \
<img src="https://drive.google.com/uc?export=view&id=11EYqpIVCPTCtsftH-7XvHFmpZVN6xe9Y" alt="Google Drive Image" width=70%/>

#### Visibility

Figure 5 shows the statistical summary of the visibility feature. For meteorology, this is the horizontal distance an object can be seen and identified in whole miles. The aviation industry uses visibility from the control tower and the pilots' visibility by looking at the runway markings. Weather changes that affect visibility in the control tower or runway will lead to disruption in airport operations, delays, or cancellations. There are some anomalous high values of visibility in the dataset, but most values are around 10 miles. These values were removed. The relationship between visibility and flight delays is complex. The systems required to support landing and take off operations under low visility depend on the airport. The ability to operate under restricted visibility such as fog also depends on the instrument rating of the pilot. In general commercial airlines in the US operate under regulations that may require a minimum of 1 mile visibility for two engine planes and  0.5 mile visibility for three ore more engines. These minimums for take-off are useful information to use this feature in our model to explain weather-related delays.

**Table 4 Statistical summary of visibility features** \
<img src="https://drive.google.com/uc?export=view&id=1S63BF8CBQVmLeQyPK5Nr5Le4Ily1Wghy" alt="Google Drive Image" width=70%/>


Figures 6 and 7 show panels overlaying certain weather features and the delays caused by weather as indicated by the flight database.
From these examples we can see the correlation between weather events  weather delays. In this case snow and precipitation in the Boston and Dallas airports during 2019. The bottom panel in both figures shows the percentage of flights delays caused by weather, while the panels above show the relevant weather feature that preceded the delays.

**Figure 6 Distribution of weather features and delays at DFW airport** \
<img src="https://drive.google.com/uc?export=view&id=11u1-6D8ZPpg3M1Zg0ueIeXOZSmR6fuN-" alt="Google Drive Image" width=70%/>

**Figure 7 Distribution of weather features and delays at Boston airport** \
<img src="https://drive.google.com/uc?export=view&id=1TknPdnYAbYIYtJ-KWC7nOb6wJwVhsLsO" alt="Google Drive Image" width=70%/>

Figure 8 shows the distribution of entries per station in a 3 month period. If a station reports their data once every hour, they should have about 2160 entries in the dataset, which is clearly not the case for the majority of the stations. A lot of them seem to report their data every 20 minutes, which is why there're a lot of stations in the 6300 - 6750 bucket. There's also a not insignificant number of stations that seem to report their data every 5 minutes. There're also a lot of stations who report with a lower frequency that once an hour on average. This may pose a challenge when deciding how to join the weather data to the flight data, as some stations may not have recent enough data, and it may not be uniform across all flights

**Figure 6 Reporting Frequency by Station**

<img src="https://drive.google.com/uc?export=view&id=1G_ZWaStDVjX1r6aa2KHMbwfOB0WJBenY" alt="Google Drive Image" width=40%/>

Figure 9 shows the pearson correlation between the previously discussed variables. As expected, all 3 temperature measurements, despite their differences, are highly positively correlated, which likely means that including more than one of them will not provide the model with more information. Humidity and Visibility also have a relatively high correlation, although negative in this case. As humidity increases, visibility decreases, as it becomes harder to see longer distances.

**Figure 7 Pearson Correlation Matrix for Hourly Weather Data**

<img src="https://drive.google.com/uc?export=view&id=1fV7nV_Wao9CCXSQ8KoP30JCbfq2Oh9gY" alt="Google Drive Image" width=40%/>

## Section 3: Machine Algorithms and Metrics

An important criteria in selecting machine learning algorithms to implement at scale is the algorithm’s parallelizability because it enables us to run our regression and classification on a large dataset with minimal runtimes. 

A common algorithm used to classification problems is K nearest neighbors (KNN). This approach uses similarity based on data points that lie nearby the sample data point. KNN employs lazy evaluation, meaning it doesn’t call the dataset until it is needed. The downside to this is that running KNN becomes computationally expensive, especially for large datasets. This can be resolved by caching, but the size of the flights and weather dataset renders this infeasible. Moreover, since the algorithm involves looking at nearby points and different centroids, for a large K, the algorithm makes many passes over the data. Therefore, KNN is not very parallelizable and will not be used.

The next approach we considered is decision trees, which is a model that starts with a single point and then branches out by decisions and their subsequent outcomes. With large datasets, we can use the MapReduce framework using each node of decisions as the split. The structure of decision trees is conducive to parallelization because we can represent each level of the tree as a MapReduce job, meaning we can represent data in a decision tree very efficiently. 

Similar to decision trees, random forests are collections of decisions trees that impart the benefits of decisions trees with additional benefits. Random forests are an ensemble operation, enabling us to test multiple hypotheses with multiple decision trees efficiently. In operation, we would take the training dataset then split it into different partition. Then each partition would have multiple decision trees mapped to different output files. We would then merge these files into the random forest. Given the efficiency at scale and expected performance, we are leaning towards using random forests. 
We will also consider neural networks (NN) and boosted trees via methods like ADA.

#####Metrics  
We selected the metrics to evaluate the performance of our models considering the business impact on our customer (the airline) due to decisions made based on incorrect predictions from our model. In addition, since we are using a classification model, we considered the ramifications on the business from false positives and false negatives. 

We considered the costs incurred by the airline attributed to delays. Airlines must bear the additional costs from their crew, costs for accommodating disrupted passengers, and the costs of aircraft re-positioning. 
There is also a cost of lost demand from potential customers who consider an airline's reputation for flight delays and customer service before purchasing. Given the competitive nature of the business on specific domestic markets, this cost may be substantial.

While our model cannot be used to mitigate the costs of most of these factors, our model can assist the airline in improving its reputation for customer satisfaction in case of delays that disrupt travel. Taking proactive measures to assist passengers may positively impact reputation and have a measurable favorable effect on sales and customer loyalty.

The consequences of our model errors in predictions can be summarised as follows:

- False positive: The model predicted a delay for the flight, but the flight departed on time. The airline incurred additional expenses to meet a demand that did not materialize. There is no impact on the airline's reputation with its customers. 

- False negative: The model predicted an on-time departure for the flight, but the flight was delayed. The airline did not allocate resources to handle the surge in demand for staff. Passengers are negatively affected by the wait time to obtain information from the airline, and the airline's reputation with its customers suffers. 

The impact of these errors depends on the number of passengers and the customer mix of the flight (share of premium passengers and economy passengers). For example, a mainline carrier operating a route with high competition, such as JFK-LAX, with many first-class and business passengers paying the highest fares, may be more interested in avoiding false negative results to retain the loyalty of premium passengers.
On the other hand, low-cost carriers operating a leisure route, such as JFK-MCO, may be more interested in reducing customer service costs and prefer a model with fewer false positives.

We can use a Fbeta score as a metric, which will allow us to consider giving more weight to either recall or precision to address the needs of our airline.

Here is the formal definition of F Beta:

<img src="https://drive.google.com/uc?export=view&id=1VqqQB1KwiL1oa3fSrygezE_xSP2hzlwC" alt="Google Drive Image" width=40%/>

#####Baseline Models

We built the following baseline model:
 - Assume `NO DELAY` classification. 
 
As part of our EDA we found that on average between 15-20% of all flights are delayed, which made us realize a very simple model that just predicted all flights would be on time could work as a good baseline for our future models.

We tested this baseline model against all of our dev sets and test set which provided us with the following results:

<img src="https://drive.google.com/uc?export=view&id=1RqmLQrCJXOhXVhkFyk3fjVw_VVc0JevR" alt="Google Drive Image" width=40%/>

One key points for these results is that we had to treat "No delays" as the positive value, rather than the delays. Since we were predicting no delays then both precision and recall would be 0, which would make this baseline useless.

#####Project Key Steps

<img src="https://drive.google.com/uc?export=view&id=1e4xOC48fziq9fNtN8--d68PaXARw4LoT" alt="Google Drive Image" width=60%/>


Block Diagram for Modeling Workflow. ( from Kurt Eulau on Slack) \
<img src="https://drive.google.com/uc?export=view&id=1nj8VFDpJvMnU7suoZCZqx5KJ8rsKD_vJ" alt="Google Drive Image" width=60%/>

## Section 4 Machine Learning Pipelines

## Section 4a: Machine Learning Joins

To join the flights and weather data, we need to figure out the closest weather station to a given airport through minimizing distance. One of the datasets provided includes information about each station and their proximity to other neighboring stations that have ICAO codes, this includes the distance to them. We noticed that a lot of these showed a disance of 0, which meant that the station was actually inside the airport.

In order to assess whether the airports in our flights dataset had a station inside them we had to join them with the station dataset. However the flights dataset only included the IATA code, and not the ICAO code. This meant that we needed an external dataset to join these. For our initial pipeline we used the dataset provided by the [Python Package index](https://pypi.org/project/airportsdata/) which not only provided both IATA and ICAO codes, but also the respective timezones for each airport which will be needed for the join between flight and weather data.

We then obtained all 388 airports in the flights dataset, by selecting the ORIGIN column and finding the distinct values within in. We then joined it with the external airport dataset to add the ICAO code. At the same time, we grouped all stations by nearest neighbour ICAO code, and found the station with the minimum distance for each of them. We then merged the closest station with the 388 airports and we found that 381 of those airports had a station with weather reports inside of them.

```
  # Find neirest stations to each airport
  closests_airport_stations = df_stations.groupBy("neighbor_call", "neighbor_id").agg({'distance_to_neighbor':'min'})

  # Get distinct airports from flight data and merge with icao
  flight_origin = df_airlines.select('origin').distinct()
  total_origin = flight_origin.count()
  flight_origin_icao = flight_origin.join(df_airport_timezone,\
                                             flight_origin.origin == df_airport_timezone.iata,\
                                             'inner').select('origin', 'icao')
```

For the remaining 7 we created a pipeline that would use their latitude and longitude to find the nearest station in KMs and miles. We found that 3 of those simply did not have a station close to them (the closest one was thousands of miles away), 2 had stations about 40 miles away, and the remaining 2 had a station about 7 miles away. We decided that only those 2 airports with relatively close station would have relevant data, while the other 5 simply did not have relevant weather data and would be dropped from the dataset.

<img src="https://drive.google.com/uc?export=view&id=1Th9jUsKXni4pEgQVJ8vk3Nz6yRDBPdCd" alt="Google Drive Image" width=50%/>

Now that we had the a station for each airport in our flights dataset we could continue with the join. However before doing that, we removed the unnecessary weather reports by dropping any reports not created from the stations airport stations. We also noticed that the flights data had duplicate entries for almost all of its flights. We dropped those duplicates as well, since they were not providing any relevant data.

The next step in our join pipeline was making sure all of the weather and flight data had equivalent timestamps. Both the flight and weather data were in their respective local timezone, however the weather data ignored daylight savings, it was not exactly the same as the flight data. We added a column to the external airport dataset to show the difference in hours with UTC for standard time in the USA for that specific reason.

All flight data would be changed from its local timezone to UTC, while all weather data would be assumed to be in the standard timezone and changed to UTC as well so all of them were in the same timezone, and they could be joined appropiately.

```
  def get_timestamp(year, month, day, hour_minutes, tz):
    hour, minutes = get_hour_minutes(hour_minutes)
    utc = timezone('UTC')
    tz = timezone(tz)
    timestamp = tz.localize(datetime(int(year), int(month), int(day), hour=int(hour), minute=int(minutes)))
    return timestamp.astimezone(utc)
    
  def get_timestamp_weather_df(date, tz):
    if date is None or tz is None:
      return 'empty'
    diff = get_utc_difference(tz)
    utc = timezone('UTC')
    timestamp = utc.localize(datetime.fromisoformat(date)) + timedelta(hours=diff)
    return timestamp.strftime("%Y%m%d%H%M")
```

In order to simplify the join, we decided to use composite keys made up of the airport's ICAO code and the timestamp, down to the hour. We did not use the minutes as the average weather report frequency is once an hour, which meant that a lot of flights would simply not coincide with a weather report. This however introduced a couple of problems. First, we could match a flight with a future weather report (eg. Flight departs at 9:01am and the Weather Report is at 9:10am, both could be matched since they are within the same 1 hour period). Second, some hours may not have weather reports for a particular airport which means those flights would not have weather data.

In order to fix the second of these problems we decided to create composite keys not only the timestamp 2 hour prior to depature (since that's when we would try to predict the delays), but 3 and 4 hours prior. This would allow us to have a longer window to find the nearest weather report to join.

In order to fix the first problem, we had 2 sets of timestamps. The one used for the composite key, which we would call the hour timestamp, and another one that would include the minutes which could be used to tell if a weather report was created prior to the flight's departure. 

After joining these datasets we could keep the one that was closest to the prediction time while still not taking place in the future, in order to avoid leakage. Out of the 42,430,592 unique flights in the original dataset, 42,227,239 were matched with a weather report that was within 3 hours prior of the prediction time. This represented 99.5% of all flights, the remaining 0.5% flights happened in airports with no stations close to them, or flights that had no weather reports in the 3 hour window.

The entire join for the complete datasets took about 1.5 hours, split into these activities:

1) Removing duplicate entries in the Flight Dataset and saving to the blob: 8.09m
2) Adding the timezone to all datasets and saving to the blob: 17.48m
3) Creating all timestamps for the flight dataset and saving to the blob: 11.27m
4) Creating all timestamps for the weather dataset and saving to the blob: 2.57m
5) Creating the composite keys and saving both datasets to the blob: 4.19m
6) Joining the flight dataset with the weather dataset for the three hours window and saving to the blob 19.3m
7) Calculcating the time difference between timesamps, using it to find the closest report and saving that to the blobl 29.09m

Total: 91.99m with a 16GB 4 core cluster.

The notebook for the join can be found here:
https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/364123876153144/command/1020093804817598

## Section 4b: Splitting the data and cross validation

Given that we are dealing with a time series we can't just randomly assign the data to either the training or test set.

Every flight from 2021 will be assigned to the blind test set, which will only be used for the final evaluation of the model.

The rest of the flights (2015 to 2020) will be used for the training set. We will conduct cross validation by implementing a blocking split. The 6 year period will be divided into 6 parts, one for each year in the dataset. Each of those parts will be split into two, the first 10 months will be used for training while the remaining 2 will be the dev set. By having the last 2 months of the year in the dev set we will avoid any data leakage.

The results of these models will be combined into a weighted average, which will be used to tune the model's hyperparameters until we find a model with satisfying results. 

```
def cross_validation_splits(df, timestamp_col, splits = 5, dev_split = 0.2):
    df = df.orderBy(col(timestamp_col)).withColumn("row_id", monotonically_increasing_id()).cache()
    n_rows = df.count()
    size_split = int(n_rows/splits)
    
    split_dataframes = []
    
    for i in range(splits):
        train_df = df.filter((col('row_id') > (size_split * i)) & (col('row_id') <= size_split * (i + 1) * (1 - dev_split)))
        dev_df = df.filter((col('row_id') > size_split * (i + 1) * (1 - dev_split)) & (col('row_id') <= size_split * (i + 1)))
        split_dataframes.append((train_df, dev_df))

    return split_dataframes

dataframes_split = cross_validation_splits(test_set, 'FLIGHT_TIMESTAMP')
```

The notebook for the split and cross-validation can be found here: 

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1020093804833045/command/1020093804833046

## Section 4c: Modelling Pipelines and results

An initial modelling pipeline we created was just a logistic regression with some of the numerical variables in our dataset. We create a pipeline that could be run for each train and dev set, and then calculate the metrics mentioned above for the results.

Since these models were so simple they had a tendency to simply predict flights to have no delays, making them perform very similarly to our baseline.

A precision of 0.820835077246851, a recall of 1, and an f2 score of 0.9581716814848401.

We expect future models that will include our manufactured variables, as well as the relevant weather data will improve the model's performance vs the baseline. 

Both the baseline and the modelling pipelines can be found in this notebook:

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1020093804825037/command/1020093804829816

Our primary task is running a binary classification model - predicting if a flight will be delayed by more than 15 minutes. We believe that the appropriate loss function is cross-entropy loss, calculated using the below formula:
$$Loss = -\frac{1}{m}\sum_{i=1}^{m}(y_{i} \cdot log(\hat{y_{i}}) + (1 - y_{i}) \cdot log(1-\hat{y_{i}}))$$

## Conclusions:

Every year flight delays cost the economy billions of dollars in lost productivity. We believe that having a model that can be used for forecast flight delays can be very attractive to airlines, airports and the travel industry in general. To build this model, we start with the hypothesis that flight departure delays can be explained by changes in weather patterns and other features associated with airport locations, origin and destination pairs, seasonal travel patterns such as holidays as well as graph features that allow to model delay propagation accross the network.

The main objective is to construct a model that can predict flight delays two hours before scheduled departure. The primary requirements to construct this model require feature engineering, feature selection and building a modeling pipeline to evaluate the perofmance of the model against a baseline model.

For phase 2, we conducted EDA on the weather and flight data, we assessed the usefulness of the weather features to explain weather delays and proposed the creation of additional features to improve our model. Given that our objective is to create a model to predict flight delays two hours prior to departure, we joined the flight and weather data sets using a composite key strategy, combining the airport code and weather report time stamp.

Once we  built a joined dataset, we produced a baseline model to compare against future more advanced models. We implemented the appropriate workflow for cross-validation of time series data.

Our efforts will continue creating our first advanced models and assessing their performance.

## Section 5: Next Steps:

We will conduct the following tasks progressively as the project advances:
    
- Feature Engineering
- Hyperparameter Tuning
- Preparation of In-Class Presentation
- Selection of Optimal Algorithm 
- Write Final Report
- Final Presentation

Links to EDA notebooks

Flight EDA

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/844424046180911/command/4295587629773808

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4295587629770423/command/4295587629773761

Weather EDA

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/364123876153045/command/4295587629773430

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/364123876153144/command/364123876153145

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1020093804820897/command/1020093804820898

## Section 6: Open Issues and Problems

We have the following open items under active discussion:

- Incorporation of additional features and transformations.
- Addressing missing values in certain features
- Introduction of graph features.